In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
df = pd.read_csv('../input/ece657a-assignment2-covid19-dataset/covid_train.csv')
df_test = pd.read_csv('../input/ece657a-assignment2-covid19-dataset/covid_test.csv')
ids = df_test['id']

In [3]:
cats = ['Client_Gender', 'Case_AcquisitionInfo', 'Reporting_PHU_City', 'Outbreak_Related']

In [4]:
df.isna().sum()

Age_Group                     6
Client_Gender                 0
Case_AcquisitionInfo          0
Reporting_PHU_City            0
Outbreak_Related           9020
Reporting_PHU_Latitude        0
Reporting_PHU_Longitude       0
Outcome1                      0
dtype: int64

In [5]:
df_test.isna().sum()

id                            0
Age_Group                     0
Client_Gender                 0
Case_AcquisitionInfo          0
Reporting_PHU_City            0
Outbreak_Related           2288
Reporting_PHU_Latitude        0
Reporting_PHU_Longitude       0
dtype: int64

In [6]:
df = df[df['Age_Group'].notna()]
df['Outbreak_Related'].fillna(value='No', inplace=True)
df_test['Outbreak_Related'].fillna(value='No', inplace=True)

df["Client_Gender"].replace(['UNSPECIFIED', 'GENDER DIVERSE'], 'OTHER', inplace = True)
df_test["Client_Gender"].replace(['UNSPECIFIED', 'GENDER DIVERSE'], 'OTHER', inplace = True)

df["Case_AcquisitionInfo"].replace(['TRAVEL', 'UNSPECIFIED EPI LINK'], 'OTHER', inplace = True)
df_test["Case_AcquisitionInfo"].replace(['TRAVEL', 'UNSPECIFIED EPI LINK'], 'OTHER', inplace = True)

# df['Reporting_PHU_City'].replace(['New Liskeard','Kingston','North Bay','Kenora','Belleville',
# 'Sault Ste. Marie','Pembroke'], 'OTHER', inplace = True) 
# df_test['Reporting_PHU_City'].replace(['New Liskeard','Kingston','North Bay','Kenora','Belleville',
# 'Sault Ste. Marie', 'Pembroke'], 'OTHER', inplace = True) 

In [7]:
df.columns

Index(['Age_Group', 'Client_Gender', 'Case_AcquisitionInfo',
       'Reporting_PHU_City', 'Outbreak_Related', 'Reporting_PHU_Latitude',
       'Reporting_PHU_Longitude', 'Outcome1'],
      dtype='object')

In [8]:
#  one hot encoding the categorical features for train and test sets
df['Age_Group'].replace(["<20", "20s", "30s", "40s", "50s", "60s", "70s", "80s", "90s"],[0,1,2,3,4,5,6,7,8], inplace = True)
df_temp = pd.get_dummies(df.iloc[:,:-1], columns = cats, drop_first=True)
df_temp['Outcome1'] = df['Outcome1']
df = df_temp

df_test['Age_Group'].replace(["<20", "20s", "30s", "40s", "50s", "60s", "70s", "80s", "90s"],[0,1,2,3,4,5,6,7,8], inplace = True)
df_test = pd.get_dummies(df_test.iloc[:,1:], columns = cats, drop_first=True)

missing_columns = [column for column in df.columns[:-1] if not column in df_test.columns]

for column in missing_columns:
    df_test[column] = 0
df_test = df_test[df.columns[:-1]]  
    
y_train_val = df['Outcome1']
X_train_val = df.drop(['Outcome1'], axis=1)
X_test = df_test

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import tree

In [10]:
param_grid = {
    'max_depth': [3, 5, 10, None],
    'n_estimators': [5, 10, 50, 150, 200]
       }

tree = RandomForestClassifier(random_state=0)
grid_search = GridSearchCV(estimator=tree, param_grid=param_grid, cv=10)
grid_search.fit(X_train_val, y_train_val)
grid_search.best_params_

{'max_depth': 10, 'n_estimators': 200}

In [11]:
clf = RandomForestClassifier(max_depth=10, n_estimators=200, random_state=0)
clf.fit(X_train_val, y_train_val)
y_pred = clf.predict(X_test)

result = pd.DataFrame(columns=['id','Outcome1'])
result['id'] = ids
result['Outcome1'] = y_pred

result.set_index(keys='id', inplace=True)
result.to_csv('mysubmissionnnn20.csv')